In [1]:
#Importing libraries
from bs4 import BeautifulSoup
import requests
import re
import urllib
import pprint
from urllib import parse
import time
import pickle

In [2]:
#Parent url
url = 'https://www.nhs.uk/servicedirectories/pages/nhstrustlisting.aspx'

In [3]:
#Creating a BeautifulSoup object out of the parent page's HTML
data = requests.get(url)
soup = BeautifulSoup(data.text, 'html.parser')

In [4]:
#Calling the linklist pickle file from earlier
linklist = pickle.load( open( "linklist.p", "rb" ) )

In [5]:
#Adding the root string to each link
fullreadlinks = []

for link in linklist:
    fullreadlinks.append('https://www.nhs.uk/'+link.replace(r"'",""))

In [6]:
#Filling lists of postcodes and names
postcodes = []
names = []
for link in fullreadlinks:
    link = requests.get(link)
    link = BeautifulSoup(link.text,'html.parser')
    if re.search("[A-Z]+[0-9]+[A-Z]?\s[0-9][A-Z][A-Z]",str(link)) != None:
        postcodes.append(re.search("[A-Z]+[0-9]+[A-Z]?\s[0-9][A-Z][A-Z]",str(link))[0])
        names.append(str(re.search("Contact details - .+ -",str(link))[0]).replace('Contact details - ','').replace(' -',''))
    else:
        print('Missing postcode') #Useful for detecting failures.
    time.sleep(0.15) #This is important! We do not want to DDOS the NHS.

In [8]:
#Storing the lists of postcodes and names as pickle objects to avoid re-running this script and reduce calls to the NHS' servers
pickle.dump(postcodes, open( "postcodes.p", "wb") )
pickle.dump(names, open( "names.p", "wb") )